In [1]:
import torch
torch.cuda.empty_cache() 

In [2]:
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2
import urllib.request
from PIL import Image

In [3]:
import pandas as pd
import pyproj

In [4]:
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [5]:
from pathlib import Path

In [6]:
import joblib

In [7]:
from mt_function_yl import *

In [8]:
import seaborn as sn

In [9]:
from scipy import stats

In [10]:
from tabulate import tabulate

In [11]:
import statsmodels.api as sm

### Data loading

In [12]:
aa = pd.read_csv('/home/yelhe/script/mt/dataraw/dfaa.csv')

In [13]:
atime = aa[['Nr', 'Datum', 'Zeit']]

In [14]:
atime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Nr      350 non-null    int64 
 1   Datum   350 non-null    object
 2   Zeit    350 non-null    object
dtypes: int64(1), object(2)
memory usage: 8.3+ KB


In [15]:
atime

,Nr,Datum,Zeit
0,99900060852597,2022-05-29,00:10
1,99900063018611,2022-10-12,17:40
2,99900052723510,2020-10-26,16:50
3,99900061514043,2022-07-08,21:30
4,99900063037843,2022-10-13,19:40
...,...,...,...
345,99900055905276,2021-06-08,12:50
346,99900060670975,2022-05-16,14:42
347,99900047367168,2019-09-21,13:49
348,99900050714619,2020-06-06,16:37


In [18]:
atime.loc[:, 'Date'] = pd.to_datetime(atime.loc[:, 'Datum'], format = "%Y-%m-%d")

/sctmp/yelhe/ipykernel_2209311/4151679598.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atime.loc[:, 'Date'] = pd.to_datetime(atime.loc[:, 'Datum'], format = "%Y-%m-%d")


In [19]:
atime.loc[:,'year'] = atime.loc[:,'Date'].dt.year
atime.loc[:,'month'] = atime.loc[:,'Date'].dt.month
atime.loc[:,'day'] = atime.loc[:,'Date'].dt.day

/sctmp/yelhe/ipykernel_2209311/2869189422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atime.loc[:,'year'] = atime.loc[:,'Date'].dt.year
/sctmp/yelhe/ipykernel_2209311/2869189422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atime.loc[:,'month'] = atime.loc[:,'Date'].dt.month
/sctmp/yelhe/ipykernel_2209311/2869189422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [20]:
atime

,Nr,Datum,Zeit,Date,year,month,day
0,99900060852597,2022-05-29 00:00:00,00:10,2022-05-29,2022,5,29
1,99900063018611,2022-10-12 00:00:00,17:40,2022-10-12,2022,10,12
2,99900052723510,2020-10-26 00:00:00,16:50,2020-10-26,2020,10,26
3,99900061514043,2022-07-08 00:00:00,21:30,2022-07-08,2022,7,8
4,99900063037843,2022-10-13 00:00:00,19:40,2022-10-13,2022,10,13
...,...,...,...,...,...,...,...
345,99900055905276,2021-06-08 00:00:00,12:50,2021-06-08,2021,6,8
346,99900060670975,2022-05-16 00:00:00,14:42,2022-05-16,2022,5,16
347,99900047367168,2019-09-21 00:00:00,13:49,2019-09-21,2019,9,21
348,99900050714619,2020-06-06 00:00:00,16:37,2020-06-06,2020,6,6


In [22]:
atime.loc[:,'week'] = atime.loc[:,'Date'].dt.isocalendar().week

/sctmp/yelhe/ipykernel_2209311/1387979514.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atime.loc[:,'week'] = atime.loc[:,'Date'].dt.isocalendar().week


In [23]:
atimegroup = atime.groupby(['year']).size().reset_index().rename(columns={0:'count'})

In [24]:
atimegroup

,year,count
0,2019,48
1,2020,74
2,2021,91
3,2022,137


In [26]:
# Traffic Count bicycle
btc19 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicycle_traffic_count_2019.csv')
btc20 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicycle_traffic_count_2020.csv')
btc21 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicycle_traffic_count_2021.csv')
# btc22
btc22 = pd.read_csv('/home/yelhe/script/mt/output/bic_tc_2022_new.csv')

# Traffic count car
vtc19 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/car_traffic_count_2019.csv')
vtc20 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/car_traffic_count_2020.csv')
vtc21 = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/car_traffic_count_2021.csv')
# vtc22
vtc22 = pd.read_csv('/home/yelhe/script/mt/output/car_tc_2022.csv')

In [35]:
vtc19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9663 entries, 0 to 9662
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MSID               9663 non-null   object 
 1   week               9663 non-null   int64  
 2   car_traffic_count  9663 non-null   int64  
 3   month              9663 non-null   int64  
 4   date               9663 non-null   object 
 5   year               9663 non-null   int64  
 6   OST                9663 non-null   float64
 7   NORD               9663 non-null   float64
 8   car_traffic_norm   9663 non-null   float64
dtypes: float64(3), int64(4), object(2)
memory usage: 679.6+ KB


In [38]:
btc22_0 = btc22.drop(['FK_STANDORT'], axis = 1)
btc22_0 = btc22_0.rename(columns = {'velo':'bicycle_traffic_count',
                                    'fk_zaehler':'FK_ZAEHLER'})

In [39]:
btc = pd.concat([btc19, btc20, btc21, btc22_0])
# btc1921 = pd.concat([btc19, btc20, btc21])
# vtc1921 = pd.concat([vtc19, vtc20, vtc21])

In [40]:
btc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7946 entries, 0 to 2169
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   FK_ZAEHLER             7946 non-null   object 
 1   week                   7946 non-null   int64  
 2   bicycle_traffic_count  7946 non-null   float64
 3   month                  7946 non-null   int64  
 4   date                   5776 non-null   object 
 5   year                   7946 non-null   int64  
 6   OST                    5776 non-null   float64
 7   NORD                   5776 non-null   float64
 8   bicycle_traffic_norm   5776 non-null   float64
dtypes: float64(4), int64(3), object(2)
memory usage: 620.8+ KB


In [41]:
vtc22_0 = vtc22.rename(columns = {'AnzFahrzeuge':'car_traffic_count'})

In [42]:
vtc = pd.concat([vtc19, vtc20, vtc21, vtc22_0])

In [43]:
vtc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40239 entries, 0 to 11465
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MSID               40239 non-null  object 
 1   week               40239 non-null  int64  
 2   car_traffic_count  40239 non-null  float64
 3   month              40239 non-null  int64  
 4   date               28773 non-null  object 
 5   year               40239 non-null  int64  
 6   OST                28773 non-null  float64
 7   NORD               28773 non-null  float64
 8   car_traffic_norm   28773 non-null  float64
dtypes: float64(4), int64(3), object(2)
memory usage: 3.1+ MB


In [71]:
atime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Nr      350 non-null    int64         
 1   Datum   350 non-null    object        
 2   Zeit    350 non-null    object        
 3   Date    350 non-null    datetime64[ns]
 4   year    350 non-null    int32         
 5   month   350 non-null    int32         
 6   day     350 non-null    int32         
 7   week    350 non-null    UInt32        
dtypes: UInt32(1), datetime64[ns](1), int32(3), int64(1), object(2)
memory usage: 16.9+ KB


In [72]:
ac_btc = atime.merge(btc, how = 'left', on = ['year', 'month', 'week'])
ac_vtc = atime.merge(vtc, how = 'left', on = ['year', 'month', 'week'])

In [46]:
# Shortest distance file
disb = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_bicyclecount.csv') 
disv = pd.read_csv('/home/yelhe/data/Traffic_network_variables/acc_carcount.csv') 

In [48]:
disbu = disb[['Nr', 'fk_zaehler', 'distance']]
disbu = disbu.drop_duplicates()

In [49]:
disvu = disv[['Nr', 'MSID', 'distance']]
disvu = disvu.drop_duplicates()

In [50]:
disbu = disbu.rename(columns = {'fk_zaehler': 'FK_ZAEHLER'})

In [73]:
len(ac_vtc['Nr'].unique())

350

In [82]:
acc_bc = disbu.merge(ac_btc, how = 'left', on = ['Nr','FK_ZAEHLER'])
acc_vc = disvu.merge(ac_vtc, how = 'left', on = ['Nr', 'MSID'])

In [85]:
len(disbu['distance'].unique())

350

In [86]:
print(len(acc_bc['Nr'].unique()), len(acc_vc['Nr'].unique()),
      len(disvu['Nr'].unique()),len(disbu['Nr'].unique()))

350 350 350 350


In [87]:
len(ac_btc['FK_ZAEHLER'].unique())

64

In [88]:
len(disbu['FK_ZAEHLER'].unique())

70

In [89]:
acc_bc.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicyclecount_new.csv')
acc_vc.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/carcount_new.csv')

In [154]:
wrbc = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bicyclecount_u.csv')
wrvc = pd.read_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/carcount_u.csv')

In [125]:
len(acc_bc['Nr'].unique())

350

In [126]:
acc_bc.index

RangeIndex(start=0, stop=518, step=1)

In [128]:
wrbc.columns

Index(['Unnamed: 0', 'Nr', 'FK_ZAEHLER', 'distance', 'Datum', 'Zeit', 'Date',
       'year', 'month', 'day', 'week', 'bicycle_traffic_count', 'date', 'OST',
       'NORD', 'bicycle_traffic_norm'],
      dtype='object')

In [129]:
wrbci = wrbc.set_index('Unnamed: 0')

In [135]:
wrbcf = acc_bc.join(wrbc,  lsuffix='', rsuffix='r',)

In [136]:
wrbcf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Nr                      518 non-null    int64         
 1   FK_ZAEHLER              518 non-null    object        
 2   distance                518 non-null    float64       
 3   Datum                   244 non-null    object        
 4   Zeit                    244 non-null    object        
 5   Date                    244 non-null    datetime64[ns]
 6   year                    244 non-null    float64       
 7   month                   244 non-null    float64       
 8   day                     244 non-null    float64       
 9   week                    244 non-null    UInt32        
 10  bicycle_traffic_count   244 non-null    float64       
 11  date                    155 non-null    object        
 12  OST                     155 non-null    float64   

In [138]:
len(wrbcf['Nr'].unique())

350

In [139]:
wrbcf.columns

Index(['Nr', 'FK_ZAEHLER', 'distance', 'Datum', 'Zeit', 'Date', 'year',
       'month', 'day', 'week', 'bicycle_traffic_count', 'date', 'OST', 'NORD',
       'bicycle_traffic_norm', 'Unnamed: 0', 'Nrr', 'FK_ZAEHLERr', 'distancer',
       'Datumr', 'Zeitr', 'Dater', 'yearr', 'monthr', 'dayr', 'weekr',
       'bicycle_traffic_countr', 'dater', 'OSTr', 'NORDr',
       'bicycle_traffic_normr'],
      dtype='object')

In [140]:
wrbcf = wrbcf[['Nr', 'bicycle_traffic_countr', 'distancer']]

In [168]:
wrbcf = wrbcf.dropna()

In [169]:
wrbcf

,Nr,bicycle_traffic_countr,distancer
0,99900060852597,11188.80428,174.563376
1,99900063018611,11188.80428,409.401533
2,99900052723510,11188.80428,180.442109
3,99900061514043,11188.80428,336.047576
4,99900063037843,11188.80428,549.328543
...,...,...,...
345,99900062477694,11188.80428,110.703046
346,99900062477694,11188.80428,310.789294
347,99900051862636,11188.80428,236.886674
348,99900061634922,11188.80428,340.463140


In [170]:
len(wrbcf['Nr'].unique())

240

In [110]:
wrbcf = wrbcf.rename(columns = {'bicycle_traffic_countr':'bicycle_traffic_count',
                               'distancer':'distance'})

In [147]:
wrbcf.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/bc_nu.csv')

In [149]:
wrvci = wrvc.set_index('Unnamed: 0')

In [155]:
wrvcf = acc_vc.join(wrvc,  lsuffix='', rsuffix='r')

In [156]:
wrvcf.columns

Index(['Nr', 'MSID', 'distance', 'Datum', 'Zeit', 'Date', 'year', 'month',
       'day', 'week', 'car_traffic_count', 'date', 'OST', 'NORD',
       'car_traffic_norm', 'Unnamed: 0', 'Nrr', 'MSIDr', 'distancer', 'Datumr',
       'Zeitr', 'Dater', 'yearr', 'monthr', 'dayr', 'weekr',
       'car_traffic_countr', 'dater', 'OSTr', 'NORDr', 'car_traffic_normr'],
      dtype='object')

In [157]:
wrvcf = wrvcf[['Nr', 'car_traffic_countr', 'distancer']]

In [163]:
wrvcf.dropna()

,Nr,car_traffic_count,distance
0,99900060852597,2644.0,799.959576
1,99900063018611,3287.0,301.504851
2,99900052723510,5032.0,119.594000
3,99900061514043,5096.0,710.541800
4,99900063037843,5731.0,476.250472
...,...,...,...
345,99900062100656,161118.0,20.397843
346,99900054491542,164644.0,192.775127
347,99900061895134,166571.0,138.487824
348,99900056387598,168260.0,15.813150


In [164]:
len(wrvcf['Nr'].unique())

350

In [165]:
wrvcf = wrvcf.rename(columns = {'car_traffic_countr':'car_traffic_count',
                               'distancer':'distance'})

In [166]:
wrvcf

,Nr,car_traffic_count,distance
0,99900060852597,2644.0,799.959576
1,99900063018611,3287.0,301.504851
2,99900052723510,5032.0,119.594000
3,99900061514043,5096.0,710.541800
4,99900063037843,5731.0,476.250472
...,...,...,...
350,99900055905276,NaN,NaN
351,99900060670975,NaN,NaN
352,99900047367168,NaN,NaN
353,99900050714619,NaN,NaN


In [121]:
wrvcf.to_csv('/home/yelhe/data/Traffic_network_variables/traffic_count/vc_nu.csv')